## App Info Scraping
We're interested in learning information about the games (genre, PEGI rating), to aid in gaining insights.

### Getting a list of unique `app_id` from the enhanced transactions dataset

In [8]:
import pandas as pd

df = pd.read_csv('OtherData/AppIdData/transactions_with_app_ids.csv')

unique_app_ids = (df[['app_id']].dropna().drop_duplicates().reset_index(drop=True))
print(unique_app_ids.info())
unique_app_ids.to_csv('OtherData/AppInfoData/unique_app_ids.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5904 entries, 0 to 5903
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   app_id  5904 non-null   object
dtypes: object(1)
memory usage: 46.3+ KB
None


### Scraping metadata from the Google Play Store

Mindlessly copying the logic from the app_id scraping notebook (library wen??)

In [9]:
from google_play_scraper import app
from google_play_scraper.exceptions import ExtraHTTPError, NotFoundError


def get_app_metadata(app_id):
    try:
        result = app(app_id, lang="en", country="us")
        return result # Return the full dict now and turn it into a dataset to merge later
    except NotFoundError as e:
        print(f"App {app_id} not found!")
    except Exception as e:
        print(f"Error searching for {app_id}: {e}")
    return None

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Use a tuple of (df_index, app_name)
def fetch_app_metadata_with_index(index, app_id):
    return index, get_app_metadata(app_id)

def run_threaded_app_metadata(df, max_workers=10):
    # This is the shared bucket for the results, every thread has its own index so no race conditions
    metadata_dict = {}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(fetch_app_metadata_with_index, idx, row['app_id']): idx
            for idx, row in df.iterrows()
        }
        for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching apps metadata"):
            idx, metadata = future.result()
            if metadata != None:
                metadata_dict[idx] = metadata

    metadata_df = pd.DataFrame.from_dict(metadata_dict, orient='index')
    df_combined = df.join(metadata_df)
    return df_combined

In [10]:
df_with_metadata = run_threaded_app_metadata(unique_app_ids)

Fetching apps metadata:  96%|█████████▌| 5655/5904 [09:55<00:28,  8.86it/s]

App com.drukhigh.ttmhigh.vip not found!


Fetching apps metadata: 100%|██████████| 5904/5904 [10:33<00:00,  9.32it/s]


In [13]:
df_with_metadata.to_csv('OtherData/AppInfoData/apps_metadata.csv', index=False)

In [12]:
# Read in full
df_with_metadata = pd.read_csv('OtherData/AppInfoData/apps_metadata.csv')
from IPython.display import display
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)       
pd.set_option('display.max_colwidth', None)
display(df_with_metadata)

app_id  \
0                       com.sbitsoft.dn94percent2   
1                     com.gabrielecirulli.app2048   
2                          kemco.hitpoint.chronus   
3                                app.airmusic.pro   
4                               com.spwatch.lumen   
...                                           ...   
5899            fun.kola.survivorsshot.oversea.yf   
5900                    com.upjers.zoo2animalpark   
5901  com.wildlife.games.battle.royale.free.zooba   
5902              com.sixtostart.zombiesrunclient   
5903                          com.zynga.livepoker   

                               title  \
0                                94%   
1                               2048   
2                    RPG Chronus Arc   
3      AirMusic - stream your music!   
4                              Lumen   
...                              ...   
5899                      Zombies.io   
5900              Zoo 2: Animal Park   
5901  Zooba: Fun Battle Royale Games   
5902  ZRX: Zombies Run + Marvel Move   
5903  Zynga Poker- Texas Holdem Game   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             